In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import descartes
import geopandas as gpd
import json
import requests
import geocoder

# Random Forest.
from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point, Polygon
#from urllib2 import urlopen
from urllib.request import urlopen #Python3


# XGBoost.
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)

In [2]:
# LECTURAS DE CSV YA PROCESADOS.
train = pd.read_csv('DATA/trainProcesado.csv')
test = pd.read_csv('DATA/testProcesado.csv')

In [3]:
Y = np.array(train['precio'])
X = train.drop('precio', axis = 1)

In [4]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
data_dmatrix_inst = xgb.DMatrix(data=trainX,label=trainY)
#X_train_inst, X_test_inst, y_train_inst, y_test_inst = train_test_split(trainX, trainY, test_size=0.25, random_state=100)
xg_reg = xgb.XGBRegressor(objective = 'reg:linear', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.05,
                          max_depth = 10, 
                          alpha = 7, 
                          n_estimators = 500)

In [6]:
xg_reg.fit(trainX,trainY)

[14:54:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [7]:
y_pred = xg_reg.predict(testX)
# Calculamos el error absoluto.
errors = abs(y_pred - testY)
# Imprimimos el error.
print('Error:', round(np.mean(errors), 2), 'grados.')

Error: 593251.05 grados.


In [8]:
# Calculamos el porcentaje de error.
mape = 100 * (errors / testY)
# Calculate la precisión.
accuracy = 100 - np.mean(mape)
print('Precision:', round(accuracy, 2), '%.')

Precision: 71.24 %.


In [9]:
prediccion = xg_reg.predict(test)

ValueError: feature_names mismatch: ['id', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'anio', 'ciudad', 'provincia', 'tipodepropiedad', 'titulo_features_enc', 'descripcion_features_enc', 'antiguedad_agrupada', 'metroscubiertos_agrupada', 'metrostotales_agrupada'] ['id', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'anio', 'ciudad', 'provincia', 'tipodepropiedad', 'titulo_features_enc_x', 'titulo_features_enc_y', 'antiguedad_agrupada', 'metroscubiertos_agrupada', 'metrostotales_agrupada']
expected descripcion_features_enc, titulo_features_enc in input data
training data did not have the following fields: titulo_features_enc_x, titulo_features_enc_y

In [ ]:
##IMPORTANCIA DE FEATURES

In [ ]:
## =================================================================================================
## ARMAMOS EN BASE A LA PREDICCIÓN QUE TENEMOS UN CSV PARA SUBIR A KAGGLE CON EL FORMATO INDICADO!
## =================================================================================================
submission = pd.DataFrame({ 'id': test['id'], 'target': prediccion })
submission.to_csv("SUBMITS/007_G34_XGBoost.csv", index=False)